<a href="https://colab.research.google.com/github/momusa/.texmf/blob/master/three_level_in_single_mode_cavity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from qutip import *
from pylab import *
import scipy

def probss(E, kappa, gamma1, gamma2, gamma12, g, wc, w31, w32, w1, wl, N, Omega):
    # construct composite operators
    ida = qeye(N)
    idatom = qeye(3)
    sig13 = Qobj([[0, 0, 1],[0, 0, 0],[0, 0, 0]])
    sig31 = Qobj([[0, 0, 0],[0, 0, 0],[1, 0, 0]])
    sig23 = Qobj([[0, 0, 0],[0, 0, 1],[0, 0, 0]])
    sig32 = Qobj([[0, 0, 0],[0, 0, 0],[0, 1, 0]])
    sig12 = Qobj([[0, 1, 0],[0, 0, 0],[0, 0, 0]])
    sig21 = Qobj([[0, 0, 0],[1, 0, 0],[0, 0, 0]])
    sig33 = Qobj([[0, 0, 0],[0, 0, 0],[0, 0, 1]])
    sig22 = Qobj([[0, 0, 0],[0, 1, 0],[0, 0, 0]])
    
    a = tensor(destroy(N), idatom)
    s13 = tensor(ida, sig13)
    s31 = tensor(ida, sig31)
    s23 = tensor(ida, sig23)
    s32 = tensor(ida, sig32)
    s12 = tensor(ida, sig12)
    s21 = tensor(ida, sig21)
    s22 = tensor(ida, sig22)
    s33 = tensor(ida, sig33)
    
    # Hamiltonian
    H = (w31 - wl) * s31*s13 + (w31 - wl - w32 + w1) * s23*s32 + (wc - wl) * a.dag() * a + \
        g * (a.dag()*s13  +  a*s31) + Omega * (s23 + s32) + E * (a.dag() + a) # E*(s31+s13) #

    # Collapse operators
    C1 = sqrt(2 * kappa) * a
    C2 = sqrt(2 * gamma1) * s13
    C3 = sqrt(2 * gamma2) * s32
    C4 = sqrt(2 * gamma12) * s12
    C1dC1 = C1.dag() * C1
    C2dC2 = C2.dag() * C2
    C3dC3 = C3.dag() * C3
    #C4dC4 = C4.dag() * C4

    #method	‘direct’, ‘eigen’, ‘power’, ‘iterative-gmres’,’iterative-lgmres’, ‘svd’
    
    # find steady state
    rhoss = steadystate(H, [C1, C2, C3, C4],use_precond=True, method='power',sparse=False,use_rcm = True, use_wbm=True,tol=1e-6)

    # calculate expectation values
    a_dag_a_e = expect(C1dC1, rhoss)
    s33_e = expect(C2dC2, rhoss)
    s22_e = expect(C3dC3, rhoss)
    s13_e = expect(s13,rhoss)
    s23_e = expect(s23,rhoss)
    s13_a_e = expect(a.dag()*s13,rhoss)
    s23_e = expect(s23,rhoss)
    a_e = expect(a, rhoss)

    return a_dag_a_e, s33_e, s22_e, s13_e, s23_e, s13_a_e, s23_e, a_e


# setup the calculation
#-----------------------
# must be done before parfunc unless we
# want to pass all variables as one using
# zip function (see documentation for an example)
kappa = 0.001
gamma1 = 0.001
gamma2 = 0.001
gamma12 =0.0
Omega = 0.1
g = 3.0
wc = 0.0
w31 = 0.0
w32 = 0.0
wl = 0
N =30
E = 0.01
nloop = 1000
w1 = linspace(-5,5,nloop)

# define single-variable function for use in parfor
# cannot be defined inside run() since it needs to
# be passed into seperate threads.
def parfunc(w1):  # function of wl only
    a_dag_a_e, s33_e, s22_e, s13_e, s23_e, s13_a_e, s23_e, a_e = probss(E, kappa, gamma1, gamma2, gamma12, g, wc, w31, w32, w1, wl, N, Omega)
    return a_dag_a_e, s33_e, s22_e, s13_e, s23_e, s13_a_e, s23_e, a_e

def run():
    # run parallel for-loop over wlist
    a_dag_a_e, s33_e, s22_e, s13_e, s23_e, s13_a_e, s23_e, a_e = parfor(parfunc, w1)
    
   # output_data=vstack((wlist,real(count1), imag(count1),real(count2), imag(count2),real(count3),imag(count3),real(sma),imag(sma), real(infield),imag(infield)))
    output_data=vstack((w1,real(a_dag_a_e),real(s33_e),real(s22_e), real(s13_e), imag(s13_e),real(s23_e), imag(s23_e),real(s13_a_e), imag(s13_a_e),real(s23_e), imag(s23_e), real(a_e),imag(a_e)))
    file_data_store('/home/photon/Dropbox/Documents/manuscripts/data/three_level_strong_cavity/quantum_model/g_3_Omeg_0_005_k_0_0001_gam1_0_0001_gam2_0_0001_gam12_0_0_E_0_040.dat',output_data.T,numtype="real",sep=",")
    
    # plot cavity emission and qubit spontaneous emssion
    fig = figure(1)
    ax = fig.add_subplot(111)
    #ax.plot(wlist, real(a_dag_a_e), lw=2)
    ax.plot(w1, real(a_dag_a_e),lw=2)
    xlabel('Drive Frequency Detuning')
    ylabel('Count rates')
    xlim(-5,5)
    ax.grid()
    show()


if __name__ == "__main__":
   run()

ModuleNotFoundError: ignored